<a href="https://colab.research.google.com/github/statisticianinstilettos/Hackathon-Assets/blob/main/ConversationalAgentWorkshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Conversational Agent Example

In this notebook, we will build a conversational Agent with CrewAI as the agent framework, and implement Opik to track your LLM calls, chains and agents. We will introduce the concept of tracing and evaluaton for conversational threads, and how to automate the evaluation of your LLM workflows.

## Getting started

We will first install the required dependencies and configure both Opik and CrewAI.

In [ ]:
%pip install -U opik crewai crewai-tools --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.2/152.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 737.5/737.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.6/472.6 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 739.6/739.6 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 5.0 MB/

In [ ]:
import os
import getpass
import opik


[Comet](https://www.comet.com/site?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) provides a hosted version of the Opik platform, [simply create an account](https://www.comet.com/signup?from=llm&utm_source=opik&utm_medium=colab&utm_content=langchain&utm_campaign=opik) and grab your API Key. You will configure your API key in the code below.


In [ ]:
# Configure Opik
opik.configure()

OPIK: Your Opik API key is available in your account settings, can be found at https://www.comet.com/api/my/settings/ for Opik cloud


Please enter your Opik API key:··········
Do you want to use "statisticianinstilettos" workspace? (Y/n)


OPIK: Configuration saved to file: /root/.opik.config
OPIK: Configuration completed successfully. Traces will be logged to 'Default Project' by default. To change the destination project, see: https://www.comet.com/docs/opik/tracing/log_traces#configuring-the-project-name


In order to configure CrewAI, you will need to have your LLM provider API key. For this example, we’ll use OpenAI. You can find or create your OpenAI API Key in this page.

In [ ]:
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


# Implementing a Conversational Agent with CrewAI Flows

The first step is to create our example project. We will create an interactive and conversational Travel Agent Planner using CrewAI flows.


CrewAI Flows is a powerful feature designed to streamline the creation and management of AI workflows. Flows allow developers to combine and coordinate coding tasks and Crews efficiently, providing a robust framework for building sophisticated AI automations.
Flows allow you to create structured, event-driven workflows. They provide a seamless way to connect multiple tasks, manage state, and control the flow of execution in your AI applications. With Flows, you can easily design and implement multi-step processes that leverage the full potential of CrewAI’s capabilities.

Let’s create a simple Flow where you will use OpenAI to generate a fun fact about a city.

In [ ]:
from crewai.flow.flow import Flow, start, listen
from opik.integrations.crewai import track_crewai

track_crewai(project_name="conversational_agent_workshop")

class ExampleFlow(Flow):
    model = "gpt-4o-mini"

    @start()
    def get_city_from_user(self):
        # Ask user for input in the notebook
        city = input("Please enter the name of a city: ")
        # Store in state for future steps
        self.state["city"] = city
        return city

    @listen(get_city_from_user)
    def generate_fun_fact(self, city):
        from litellm import completion
        response = completion(
            model=self.model,
            messages=[{"role": "user", "content": f"Tell me a fun fact about {city}"}],
        )
        fun_fact = response["choices"][0]["message"]["content"]
        # Store in state
        self.state["fun_fact"] = fun_fact
        return fun_fact


In [ ]:
import asyncio

flow = ExampleFlow()

# Use asyncio to run inside Jupyter
result = await flow.kickoff_async()

print(result)

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: ExampleFlow                                                                                              │
│  ID: aced7c00-bb0a-4cf4-acab-be6c773edbcf                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Flow started with ID: aced7c00-bb0a-4cf4-acab-be6c773edbcf

Output()

Please enter the name of a city: Austin


╭──────────────────────────────────────────────── Flow Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Flow Execution Completed                                                                                       │
│  Name: ExampleFlow                                                                                              │
│  ID: aced7c00-bb0a-4cf4-acab-be6c773edbcf                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── Execution Traces ────────────────────────────────────────────────╮
│                                                                                                                 │
│  🔍 Detailed execution traces are available!                                                                    │
│                                                                                                                 │
│  View insights including:                                                                                       │
│    • Agent decision-making process                                                                              │
│    • Task execution flow and timing                                                                             │
│    • Tool usage details                                                                                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Would you like to view your execution traces? [y/N] (20s timeout): y


╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: 81388494-a40b-4bb9-ad6f-fee7f93c87f6                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/81388494-a40b-4bb9-ad6f-fee7f93c87f6?access_code=TRA │
│ CE-ab81245670                                                                                                   │
│ 🔑 Access Code: TRACE-ab81245670                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────── 🔍 Execution Trace Generated ──────────────────────────────────────────╮
│                                                                                                                 │
│  🎉 Your First CrewAI Execution Trace is Ready!                                                                 │
│                                                                                                                 │
│  View your execution details here:                                                                              │
│  https://app.crewai.com/crewai_plus/ephemeral_trace_batches/81388494-a40b-4bb9-ad6f-fee7f93c87f6?access_code=T  │
│  RACE-ab81245670                                                                                                │
│                                                                                                                 │
│  This trace shows:                                                                                              │
│  • Agent decisions and interactions                                                                             │
│  • Task execution timeline                                                                                      │
│  • Tool usage and results                                                                                       │
│  • LLM calls and responses                                                                                      │
│                                                                                                                 │
│  ✅ Tracing has been enabled for future runs! (CREWAI_TRACING_ENABLED=true added to .env)                       │
│  You can also add tracing=True to your Crew(tracing=True) / Flow(tracing=True) for more control.                │
│                                                                                                                 │
│  📝 Note: This link will expire in 24 hours.                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Austin, Texas, is known for its vibrant live music scene and is often referred to as the "Live Music Capital of the World." The city hosts more than 250 live music venues and has a rich musical heritage that includes genres like country, rock, blues, and indie. One of the biggest events celebrating this culture is the South by Southwest (SXSW) festival, which brings together musicians, filmmakers, and tech innovators from around the globe every spring.


After running your CrewAI application, visit the Opik app to view your traces.

The track_crewai integration automatically tracks token usage and cost for all supported LLM models used during CrewAI agent execution.

Cost information is automatically captured and displayed in the Opik UI, including:

- Token usage details

- Cost per request based on model pricing

- Total trace cost